In [2]:
!pip install sparse_dot_topn

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("andrewmvd/spotify-playlists")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/andrewmvd/spotify-playlists/versions/1


In [4]:
import os
import shutil

# Define the destination directory
destination_dir = "/content/spotify-playlists"

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Move the files to the new location
for filename in os.listdir(path):
    source_path = os.path.join(path, filename)
    destination_path = os.path.join(destination_dir, filename)
    shutil.move(source_path, destination_path)

print("Files moved to:", destination_dir)

Files moved to: /content/spotify-playlists


In [5]:
import pandas as pd
import csv

df = pd.read_csv(
    '/content/spotify-playlists/spotify_dataset.csv',
    engine='python',             # Use the Python engine for more flexibility
    quoting=csv.QUOTE_NONE,      # Treat quotes as regular characters
    on_bad_lines='skip'          # Skip problematic lines (or use 'warn' for warnings)
)

In [6]:
df = df[[' "trackname"', ' "playlistname"']]
df.columns = ['trackname', 'playlistname']
df

,trackname,playlistname
0,"""(The Angels Wanna Wear My) Red Shoes""","""HARD ROCK 2010"""
1,"""7 Years Too Late""","""HARD ROCK 2010"""
2,"""Accidents Will Happen""","""HARD ROCK 2010"""
3,"""Alison""","""HARD ROCK 2010"""
4,"""All Be Okay""","""HARD ROCK 2010"""
...,...,...
12199334,"""Wild Side""","""iPhone"""
12199335,"""Woman""","""iPhone"""
12199336,"""You Don't Know How It Feels""","""iPhone"""
12199337,"""You Wreck Me""","""iPhone"""


In [7]:
unique_tracks = df['trackname'].nunique()
unique_playlists = df['playlistname'].nunique()

print(f"Number of unique tracks: {unique_tracks}")
print(f"Number of unique playlists: {unique_playlists}")

Number of unique tracks: 1854175
Number of unique playlists: 152331


In [8]:
# Count tracks appearing at least twice
track_counts = df['trackname'].value_counts()
tracks_appearing_at_least_twice = track_counts[track_counts >= 2].count()

print(f"Number of tracks appearing at least twice: {tracks_appearing_at_least_twice}")

Number of tracks appearing at least twice: 872129


In [9]:
# Get the rows of tracks that appear at least twice
simplex_1 = df[df['trackname'].isin(track_counts[track_counts >= 2].index)]
simplex_1

,trackname,playlistname
0,"""(The Angels Wanna Wear My) Red Shoes""","""HARD ROCK 2010"""
2,"""Accidents Will Happen""","""HARD ROCK 2010"""
3,"""Alison""","""HARD ROCK 2010"""
4,"""All Be Okay""","""HARD ROCK 2010"""
5,"""Band On The Run""","""HARD ROCK 2010"""
...,...,...
12199334,"""Wild Side""","""iPhone"""
12199335,"""Woman""","""iPhone"""
12199336,"""You Don't Know How It Feels""","""iPhone"""
12199337,"""You Wreck Me""","""iPhone"""


In [ ]:
# from collections import defaultdict
# from itertools import combinations
# from tqdm import tqdm

# playlist_song_pairs = defaultdict(int)
# for playlist_name, playlist_data in tqdm(simplex_1.groupby('playlistname')):
#     tracks = playlist_data['trackname'].tolist()

#     for pair in combinations(tracks, 2):
#         pair = tuple(sorted(pair))  # Ensure consistent order
#         playlist_song_pairs[pair] += 1

  1%|          | 1226/150232 [00:48<1:38:25, 25.23it/s] 


KeyboardInterrupt: 

Co-occurrence matrices again

Construct a binary matrix of rows with playlistnames and cols of trackname. If a track appears in a playlist, then row playlist, col trackname will be 1, else 0.

Now if we take the transpose and dot product, we get a co-occurrence matrix.

1. Transpose is a matric with rows trackname and cols playlistnames. Same concept
2. When we take the dot product, the row of the transpose is pairwise multiplied and summed.
3. The dot product second term is the original binary matrix
4. So in essence we are multiplying a row of 1s and 0s for if a particular track appeared in a playlist (from the transpose) with a columns of if a particular track appeared in a playlist (from original matrix)
5. So for each pair of tracks, we are looking at what playlists they appeared in. If a track appeared in the same playlist, they will both have a 1. If not, at least one will be 0. When these are summed up, the 1s will add up and give you how many playlists the 2 tracks appeared in
6. The final matrix will be a matrix of trackname by trackname. Element ij will be how many times track i and track j appeared in a playlist together.

In [ ]:
# import pandas as pd
# from scipy.sparse import csr_matrix

# # Build a binary matrix where rows are playlists and columns are tracks.
# playlist_track = pd.crosstab(
#     simplex_1['playlistname'],
#     simplex_1['trackname'],
#     sparse=True
# )

# # Convert to a sparse matrix (efficient if the matrix is mostly zeros).
# X = csr_matrix(playlist_track.values)

# # Compute the co-occurrence matrix: X.T * X gives, for each pair of tracks,
# # the number of playlists in which they appear together.
# co_occurrence = X.T.dot(X)

# # Optionally, convert the sparse matrix back to a DataFrame for easy lookup.
# # co_occurrence_df = pd.DataFrame(
# #     co_occurrence.toarray(),
# #     index=playlist_track.columns,
# #     columns=playlist_track.columns
# # )

# # co_occurrence_df.loc[a, b] now holds the count of playlists where 'a' and 'b' co-occur.

TypeError: crosstab() got an unexpected keyword argument 'sparse'

In [10]:
song_counts = simplex_1['trackname'].value_counts()

song_counts[song_counts >= 2]

,count
trackname,
"""Intro""",6550
"""Home""",5506
"""Closer""",3500
"""Runaway""",3291
"""Hold On""",3187
...,...
"""Ripmnp""",2
"""Return 99""",2
"""Que hace una chica como tu - directo""",2


Below pd.factorize converts string playlistname and trackname to integers based on the rows in simplex_1.

We factorize each column and get an in order integer list representing which playlist/trackname they represent and the unique ints representing them

Create a data array with np.ones the length of simplex_1 rows. This represents that every row in simplex_1 is an instance of a track being present in a playlist

Create a coo matrix = coordinate format matrix. Basically a sparse matrix with values specified by coordniates. We want it to be shape of length unique playlist by length unique tracks.

The data is the list of ones representing when a track was in a playlist. The coordinates for these 1s are given by playlist_codes and track_codes.

In [11]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix

# Convert playlist and track names to integer codes
playlist_codes, playlist_uniques = pd.factorize(simplex_1['playlistname'])
track_codes, track_uniques = pd.factorize(simplex_1['trackname'])

# Create the data array; each occurrence is counted as 1
data = np.ones(len(simplex_1), dtype=np.int32)

# Build the sparse matrix (rows: playlists, columns: tracks)
playlist_track_sparse = coo_matrix(
    (data, (playlist_codes, track_codes)),
    shape=(len(playlist_uniques), len(track_uniques))
)

# Optionally convert to CSR format for efficient arithmetic operations:
playlist_track_sparse = playlist_track_sparse.tocsr()

In [ ]:
simplex_1.to_csv('simplex1.csv', index=False)

In [12]:
del simplex_1
del df

In [13]:
import numpy as np

# Calculate the percentage of empty entries in the sparse matrix
total_elements = playlist_track_sparse.shape[0] * playlist_track_sparse.shape[1]
non_zero_elements = playlist_track_sparse.nnz  # Number of non-zero elements

empty_percentage = ((total_elements - non_zero_elements) / total_elements) * 100

print(f"Shape: {playlist_track_sparse.shape}")
print(f"Percentage of empty entries in playlist_track_sparse: {empty_percentage:.2f}%")

Shape: (150232, 872129)
Percentage of empty entries in playlist_track_sparse: 99.99%


In [ ]:
# co_occurrence = playlist_track_sparse.T.dot(playlist_track_sparse)

In [14]:
from sparse_dot_topn import sp_matmul_topn
import multiprocessing

n_threads = multiprocessing.cpu_count()

# Adjust these parameters based on your data characteristics:
top_n = 10           # keep only the top 10 values per row
threshold_value = 2  # only retain entries with a value of at least 2
density_estimate = 0.01  # lower this if your expected density is very low

co_occurrence_thresholded = sp_matmul_topn(
    playlist_track_sparse.T,
    playlist_track_sparse,
    top_n=top_n,
    threshold=threshold_value,
    n_threads=n_threads,
    density=density_estimate
)

In [16]:
import numpy as np
from scipy.sparse import save_npz

# Save the sparse matrix to a file
save_npz('co_occurrence_thresholded.npz', co_occurrence_thresholded)

print("co_occurrence_thresholded saved to co_occurrence_thresholded.npz")

co_occurrence_thresholded saved to co_occurrence_thresholded.npz
